In [65]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [66]:
#RMSLE
import timeit
import math
def rmsle(y, y_pred):
    assert len(y) == len(y_pred)
    terms_to_sum = [(math.log(y_pred[i] + 1) - math.log(y[i] + 1)) ** 2.0 for i,pred in enumerate(y_pred)]
    return (sum(terms_to_sum) * (1.0/len(y))) ** 0.5

In [ ]:
# Make predictions fit for Kaggle
# No negatives
def fit_for_kaggle(y):
    for i in range(len(y)):
    y [i] = max(0,y[i])
# Only int
y = y.astype(int)

In [67]:
#import all data
feature_data = pd.read_csv('kaggle_data/features.txt', header=None, sep="  ", names=['feature_names', 'feature_description'], engine='python')
list_feature_names = list(feature_data['feature_names'])
train_data = pd.read_csv('kaggle_data/train.csv', header=None, sep=" ", names=list_feature_names)
test_data=pd.read_csv('kaggle_data/test-val.csv',header=None,sep=" ",names=list_feature_names)
target_data = pd.read_csv('kaggle_data/train-targets.csv', sep=",")

In [68]:
#Data engineering 

# 'weekday' encoding
weekday_data = pd.get_dummies(train_data['weekday'],prefix='weekday',drop_first=True)

# 'category' encoding
category_data = pd.get_dummies(train_data['category'],prefix='category',drop_first=True)

#fusion that shit
other_data = train_data.drop(['weekday','category'],axis=1)
training_data = pd.concat([category_data,weekday_data,other_data],axis=1)

In [69]:
#lets do the same with the test data
weekday_data_test=pd.get_dummies(test_data['weekday'],prefix='weekday',drop_first=True)
category_data_test=pd.get_dummies(test_data['category'],prefix='category',drop_first=True)
other_data_test=test_data.drop(['weekday','category'],axis=1)
testing_data=pd.concat([category_data_test,weekday_data_test,other_data_test],axis=1)

#see it
#testing_data.head(5)


In [70]:
#Data standardization
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
scaler.fit(training_data)
X_reg = scaler.transform(training_data)
y_reg = target_data['Prediction'].values

#scale that test data
X_test = scaler.transform(testing_data)

#visualize X_reg
#print X_reg

In [71]:
# Making the kfold
from sklearn import model_selection
kf = model_selection.KFold(n_splits = 10)
k_folds = kf.split(X_reg, y_reg)

In [72]:
# Setting up the Linear regression
from sklearn import linear_model
lin_reg = linear_model.LinearRegression()

In [73]:
# Predictions by Cross validation
y_pred = model_selection.cross_val_predict(lin_reg, X_reg, y_reg, cv=k_folds)
# num shares can't be negative, so 0 instead
fit_for_kaggle(y_pred)

In [74]:
# Let's see the error
from sklearn import metrics
print("Mean squared error: %.3f" % metrics.mean_squared_error(y_reg, y_pred))

Mean squared error: 60429191.508


In [75]:
#RMSLE 
rmsle(y_reg, y_pred)

1.2177093989935257

In [99]:
# Prediction on Test set
lin_reg.fit(X_reg,y_reg)
y_test_reg = lin_reg.predict(X_test)
fit_for_kaggle(y_test_reg)
print(y_test_reg)

[3159 4992 1855 ..., 2680 1919 3575]


In [100]:
# Exporting
data_out = pd.DataFrame(y_test_reg, columns = ['Prediction'])
data_out.index.name = 'Id'
data_out.head(5)
data_out.to_csv("solution_reg.csv", sep = ",")